In [1]:
import gff3_parser
import polars as pl
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

orfs = pl.from_pandas(gff3_parser.parse_gff3("input/mapping_orf_Scer_SGD_noMT.gff", parse_attributes=True))

GFF = gff3_parser.parse_gff3("input/Scer.gff", parse_attributes=True)

FASTA = list(SeqIO.parse("input/Scer.fna", "fasta"))


COLUMNS = [["seq_id", "start", "end", "strand", "phase", "attributes"]]

 Input genomic fasta file: Scer_SGD.fna
 Input gff file: Scer_SGD.gff
Building structured data...


100%|██████████| 351379/351379 [00:01<00:00, 260564.78it/s]


Adding Supplemental Attribute table...
Finding unique attribute keys...


100%|██████████| 351377/351377 [00:01<00:00, 297786.79it/s]


Making attribute table...


...

100%|██████████| 23076/23076 [00:00<00:00, 223505.60it/s]


Adding Supplemental Attribute table...
Finding unique attribute keys...


100%|██████████| 23058/23058 [00:00<00:00, 216074.78it/s]


Making attribute table...


100%|██████████| 23058/23058 [00:00<00:00, 84300.97it/s]


In [6]:
from collections import OrderedDict as OD
import polars as pl
import warnings
import re

GFF_POLARS = pl.from_pandas(GFF)

ORF_DF_COLUMNS = ['Seqid',
 'Source',
 'Type',
 'Start',
 'End',
 'Score',
 'Strand',
 'Phase',
 'Status',
 'color',
 'Parent',
 'ID',
 'Ovp_with',
 'Ovp_gene']


class GenomicFeature:

    def __init__(self, ID, start, end, gene=None):
        self.ID = ID
        self.start = start
        self.end = end
        self.gene = gene
        self.counter = 0



class Gene(GenomicFeature):

    counter = 0

    def __init__(self, ID, chromosome, start, end, multi, sense, frame):
        super().__init__(ID, start, end)
        self.chromosome = chromosome
        self.frame = frame
        self.sense = sense
        self.multi = multi
        self.aORFs = OD()
        self._exons = OD()
        Gene.counter += 1
        

    @property
    def exons(self):
        return self._exons

    @exons.setter
    def exons(self, value):
    
        self._exons = value

    def add_exon(self, key, value):
        
        if key in self._exons:
            raise KeyError(f'exon {key} already exists')

        elif type(value) != list:

            raise TypeError('exon must be a list')
        
        else:
            if self.sense == "+":
                self._exons[key] = value

            elif self.sense == "-":
                self._exons[key] = value[::-1]

    def add_orf(self, key, value):

        if type(value) != Orf:
            raise TypeError('value must be Orf')

        elif key in self.aORFs:
            raise KeyError(f'orf {key} already exists')
        else:
            self.aORFs[key] = value

    
    def sort_exons(self):

        if self.sense == "+":
            self.exons = OD(sorted(self.exons.items(), key=lambda x: x[1][0]))

        elif self.sense == "-":
            self.exons = OD(sorted(self.exons.items(), key=lambda x: x[1][0], reverse=True))

class Exon(GenomicFeature):

    def __init__(self, ID, gene, start, end, abs_frame):
        super().__init__(ID, start, end, gene)
        self.abs_frame = abs_frame
 
class Orf(GenomicFeature):

    def __init__(self, ID, start, end, gene):
        super().__init__(ID, start, end, gene)
        self._ribospike = None
        self._ribostart = None
        self._frame = None

    @property
    def ribospike(self):
        return self._ribospike
    
    @ribospike.setter
    def ribospike(self, value):
        if type(value) != int:
            raise TypeError('ribospike must be int')
        else:
            self._ribospike = value
            self._ribostart = self.start + self._ribospike
            
    @property
    def frame(self):
        return self._frame
    
    def get_ribodata(self):

        pass

    
class GeneStructureError(Exception):
    def __init__(self, message):
        self.message = message
        super().__init__(self.message)


def get_good_column_names(columns):

    import re
    from itertools import chain

    parent_col = [col for col in columns if re.match(r"[Pp]arent", col)]
    name_col = [col for col in columns if re.match(r"[Nn]ame", col)]

    if len(parent_col) == 1 and len(name_col) == 1:

        return str(parent_col[0]), str(name_col[0])
    
    else:

        raise AttributeError("Problem with GFF columns : Parent or Name not found. See get_good_column_names()")
        
def return_gene_infos(gene_infos, all_exons = True) -> dict:

    chromosome = gene_infos["Seqid"].unique().to_list()[0],
    start = gene_infos.filter(pl.col("Type") == "gene")["Start"].to_list()[0],
    end = gene_infos.filter(pl.col("Type") == "gene")["End"].to_list()[0],
    sense = gene_infos.filter(pl.col("Type") == "gene")["Strand"].to_list()[0]
    multi = gene_infos["Type"].to_list().count("CDS") > 1


    return {

        "chromosome" : chromosome,
        "start" : start,
        "end" : end,
        "sense" : sense,
        "multi" : multi
    } 

def init_gene_object(gene_id, gff_dataframe):


    """
    
    On veut pour la gene_id donnée initiliaser un objet Gene avec ses attributs
    tirés du gff_dataframe.
    
    """

    pattern = fr".*{gene_id}.*"

    parent, name = get_good_column_names(gff_dataframe.columns)

    # Polars does not support regex filtering : pandas is used instead
    gene_rows = pl.from_pandas(gff_dataframe[
        gff_dataframe[parent].str.contains(pattern, regex=True, na=False) |
        gff_dataframe[name].str.contains(pattern, regex=True, na=False)
    ]) # Get all rows related to the gene being initialized


    
    gene_infos = return_gene_infos(gene_rows)


    gene = Gene(

        ID = gene_id,
        chromosome = gene_infos["chromosome"],
        start = gene_infos["start"],
        end = gene_infos["end"],
        sense = gene_infos["sense"],
        multi = gene_infos["multi"],
        frame = gene_infos["start"] % 3
        
    )
    

    cds_counter = 1
    for exon in gene_rows.filter(pl.col("Type") == "CDS").iter_rows(named = True):

        key = f'{exon["ID"]}-{cds_counter}' if exon["ID"] else f'{exon["Name"]}-{cds_counter}'
        
        gene.add_exon(
            key = key,
            value = [exon["Start"], exon["End"]]
        )

        cds_counter = cds_counter + 1

    gene.sort_exons()

    return gene

def check_double_overlap(row : tuple):

    orf = dict(zip(ORF_DF_COLUMNS, row))
    
    overlaps = [match for item in orf["Ovp_with"].split("|") for match in re.findall(r"\b([\w-]+)_mRNA\b", item)]

    if len(overlaps) == 0:

        overlaps = [match for item in orf["Ovp_with"].split("|") for match in re.findall(r"\b([\w-]+)_CDS\b", item)]
    
    buffer = []

    if len(overlaps) != 1:

        for overlap in overlaps:

            
            if GFF_POLARS.filter(
                
                (pl.col("ID") == overlap)
                )["Strand"].unique().to_list()[0] == orf["Strand"]:

                buffer.append(overlap)
                

        if len(buffer) == 1: # Several genes are overlapped by the ORF, but only one is on the same strand

            orf["Ovp_gene"] = buffer[0]
            return tuple(orf.values())
        
    
        elif len(buffer) == 0: # No gene found on the same strand as the ORF

            orf["Ovp_gene"] = "NA"
            return tuple(orf.values())
        
        else: # Several genes found on the same strand as the ORF

            orf["Ovp_gene"] = "Two_or_more_genes"
            return tuple(orf.values())


    else: # If there is only one gene found in the overlapping information given by ORFMine ID

        orf["Ovp_gene"] = overlaps[0]
        return tuple(orf.values())



True

Décider de la structure de de chaque ORF/Exon + de la structure qui les contient 
Comment implémenter ?


In [3]:
pattern = r'\b([\w-]+)_CDS\b'

same_CDS_dframe = (
    orfs
    .filter(pl.col("Type") == "nc_ovp_same-CDS")
    .with_columns([
        pl.col("ID").apply(lambda value: value.split("_")[3]).alias("Phase"), # Extract absolute strand phase from the ID generated by ORFMine
        pl.lit("NA").alias("Ovp_gene") # Create a column for the gene ID that will be filled by check_double_overlap()
    ])
)


same_CDS_dframe = same_CDS_dframe.select(ORF_DF_COLUMNS).apply(check_double_overlap) # Use .apply() method to leverage parallelization

same_CDS_dframe.columns = ORF_DF_COLUMNS

grouped = same_CDS_dframe.groupby("Ovp_gene")





In [7]:
gene_list = list()

for overlapped_feature_name , data in grouped: # This loop returns the name by which data is groupped, and the data itself as a polars dataframe

    if "gene" in GFF_POLARS.filter(pl.col("ID") == overlapped_feature_name)["Type"].unique().to_list(): # If overlapped feature is not a gene ( = transposable for example ) it's not stored

        gene = init_gene_object(overlapped_feature_name, GFF)
   

        for row in data.iter_rows(named = True):

            orf = Orf(ID = row["ID"],
            start = row["Start"],
            end = row["End"],
            gene = gene)

            gene.add_orf(key = orf.ID, 
                         value = orf)

        gene_list.append(gene)



KeyboardInterrupt: 

In [9]:
from collections import OrderedDict

nested_dict = OrderedDict()

nested_dict['outer_key1'] = OrderedDict()
nested_dict['outer_key1']['inner_key1'] = OrderedDict()
nested_dict['outer_key1']['inner_key1']['inner_inner_key1'] = 'value1'
nested_dict['outer_key1']['inner_key1']['inner_inner_key2'] = 'value2'



In [12]:
from collections import OrderedDict

ordered_dict = OrderedDict()

ordered_dict['key1'] = ['value1', 'value2', 'value3']
ordered_dict['key2'] = ['value4', 'value5']
ordered_dict['key3'] = ['value6']

for value in ordered_dict.values():

    print(value)

['value1', 'value2', 'value3']
['value4', 'value5']
['value6']
